# TIL Code Remake
### R-MNIST Dataset

In [ ]:
from os.path import isdir
from os import mkdir
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets
from tqdm import tqdm

In [ ]:
device_id = 0
torch.cuda.set_device(device_id)
device = 'cuda:' + str(device_id)
print(device)

In [ ]:
datapath = './datasets'

if not isdir(datapath):
    mkdir(datapath)

mnist_train = datasets.MNIST(datapath, train=True, download=True)
x_train = mnist_train.data
y_train = mnist_train.targets
mnist_test = datasets.MNIST(datapath, train=False, download=True)
x_test = mnist_test.data
y_test = mnist_test.targets

print('train x data before', x_train.size())
print('test x data before', x_test.size())

x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

print('\ntrain x data after', x_train.size())
print('test x data after', x_test.size())

print('\ntrain y', y_train.size())
print('test y', y_test.size())

In [ ]:
# allows the randomization to go through different seeds
num_task = 20
step = 100
init = 100
seeds = np.arange(init, num_task*step + init, step)

# finds how many classes there are and makes an empty train_y
n_class = len(np.unique(y_train))

# changes each number from the labels in y_train
# into 1's for the column in train_y that corresponds with the label
# this allows train_y to have the labels encoded into 0's and 1's only
train_y = np.zeros((len(y_train), n_class))
for i in range(train_y.shape[0]):
    train_y[i, y_train[i]] = 1

# same thing done on test_y
test_y = np.zeros((len(y_test), n_class))
for i in range(test_y.shape[0]):
    test_y[i, y_test[i]] = 1

# Check shape of train and test y
print('empty train y', train_y.shape)
print('empty test y', test_y.shape)

data = [[x_train, train_y]]
data_test = [[x_test, test_y]]

pixels = np.arange(x_train.shape[1])
pixel_set = [pixels]

for task in range(num_task - 1):
    np.random.seed(seeds[task])
    np.random.shuffle(pixels)
    pixel_set.append(pixels)

    x = x_train[:, pixels]
    data.append([x, train_y])

    x = x_test[:, pixels]
    data_test.append([x, test_y])

X_train, X_test = [], []
Y_train, Y_test = [], []

for task in range(num_task):
    X_currTask_train = data[task][0].numpy()
    Y_currTask_train = data[task][1]
    X_currTask_test = data_test[task][0].numpy()
    Y_currTask_test = data_test[task][1]

    X_train.append(X_currTask_train)
    X_test.append(X_currTask_test)
    Y_train.append(Y_currTask_train)
    Y_test.append(Y_currTask_test)

X_train = np.asarray(X_train)
X_train = np.reshape(X_train, (num_task, X_train.shape[1], X_train.shape[2]))
X_test = np.asarray(X_test)
X_test = np.reshape(X_test, (num_task, X_test.shape[1], X_test.shape[2]))
Y_train = np.asarray(Y_train)
Y_train = np.reshape(Y_train, (num_task, Y_train.shape[1], Y_train.shape[2]))
Y_test = np.asarray(Y_test)
Y_test = np.reshape(Y_test, (num_task, Y_test.shape[1], Y_test.shape[2]))

print('train X', X_train.shape)
print('test X', X_test.shape)
print('train Y', Y_train.shape)
print('test Y', Y_test.shape)